## Gather

In [53]:
import api_key
client_access_token=api_key.client_access_token
import requests
import time
from pathlib import Path
import os

In [3]:
from lyricsgenius import Genius

In [59]:
def GatherLyrics(song_name,artist_name,file_name):
    genius=Genius(client_access_token)
    song=genius.search_song(song_name,artist_name)
    vec=song.lyrics.split("\n")
    
    file=open(file_name,"a")
    file.write(">>>"+song_name+" "+artist_name+"\n")
    file.write("<sos>\n")
    for line in vec:
        file.write(line+"\n")
    file.write("<eos>\n")
    file.close()
    
    subpath="./lyrics/"+artist_name
    if not os.path.exists(subpath):
        os.makedirs(subpath)
        
    subfile=open(Path(subpath+"/"+song_name+".txt"),"w")
    for line in vec:
        subfile.write(line+"\n")
    subfile.close()
    
    print("Finish gathering "+song_name+" by "+artist_name)
    

In [60]:
GatherLyrics("Blank Space","Taylor Swift", "lyrics.txt")

Searching for "Blank Space" by Taylor Swift...
Done.
Finish gathering Blank Space by Taylor Swift


In [29]:
import requests
from bs4 import BeautifulSoup

In [48]:
def GatherSongNames(url):
    response=requests.get(url)
    names=[]
    if(response.status_code==200):
        html_string=response.text
        document=BeautifulSoup(html_string,"html.parser")
        tag_value=65
        tag=document.find(lambda tag: tag.name=="h3" and tag.text=="0–9")
        while not tag and tag_value<90:
            tag=document.find(lambda tag: tag.name=="h3" and tag.text==chr(tag_value))
            tag_value+=1
        if tag:
            tags=tag.find_all_next(lambda tag: tag.name=="a" and "title" in tag.attrs and len(tag.attrs)==2)
            skip=False
            for tag in tags:
                if tag.text=="Categories" or tag.text=="previous page" or tag.text.startswith("Template"):
                    skip=True
                if tag.text=="next page":
                    skip=True
                    new_link="https://en.wikipedia.org/"+tag["href"]
                    names.extend(GatherSongNames(new_link))
                if not skip:
                    names.append(tag.text.split(" (")[0])
    return names
def GatherSongNamesByArtist(artist_name):
    wiki_link_template="https://en.wikipedia.org/wiki/Category:"
    wiki_link=wiki_link_template+artist_name.replace(" ","_")+"_songs"
    wiki_link_2=wiki_link_template+artist_name.replace(" ","_")+"_(musician)_songs"
    names=[]
    
    names.extend(GatherSongNames(wiki_link))
    if not names:
        names.extend(GatherSongNames(wiki_link_2))
    return names

In [86]:
print(GatherSongNamesByArtist("Tyler Childers"))

['In Your Love']


In [68]:
artist_list=["Taylor Swift","Morgan Wallen","Luke Combs","Olivia Rodrigo","Post Malone","Zach Bryan","Dua Lipa",\
             "Ed Sheeran","The Weeknd","Miley Cyrus","Doja Cat","Kendrick Lamar","Harry Styles","Billie Eilish",\
            "Jason Aldean","Bad Bunny","Lana Del Rey","Bruno Mars","Tyler Childers","Adele","Rihanna","Imagine Dragons"]

In [84]:
artist_list1=["Selena Gomez","Mac Miller","Tyler Childers"]

In [64]:
import random

In [70]:
def GatherCorpus(artist_list,file_name,from_artist,from_song):
    
    global current_artist
    global current_song
    
    if from_artist==None:
        for artist in artist_list:
            
            current_artist=artist
            
            songs=GatherSongNamesByArtist(artist)
            time.sleep(1)
            print("Gathering lyrics from "+artist)
            for song in songs:
                
                current_song=song
                
                GatherLyrics(song,artist,file_name)
                time.sleep(random.random()*2)
    else:
        skip_artist=True
        for artist in artist_list:
            if skip_artist:
                if artist!=from_artist:
                    continue
                else:
                    skip_artist=False
                    
            current_artist=artist
            
            songs=songs=GatherSongNamesByArtist(artist)
            time.sleep(1)
            print("Gathering lyrics from "+artist)
            if artist==from_artist:
                skip_song=True
                for song in songs:
                    if skip_song:
                        if song!=from_song:
                            continue
                        else:
                            skip_song=False
                            continue
                            
                    current_song=song
                    
                    GatherLyrics(song,artist,file_name)
                    time.sleep(random.random()*2)     
            else:
                for song in songs:
                    
                    current_song=song
                    
                    GatherLyrics(song,artist,file_name)
                    time.sleep(random.random()*2)

In [85]:
current_artist=None
current_song=None
try:
    GatherCorpus(artist_list1,"lyrics.txt",None,None)
except:
    time.sleep(random.random()*10)
    GatherCorpus(artist_list1,"lyrics.txt",current_artist,current_song)

Gathering lyrics from Selena Gomez
Searching for "999" by Selena Gomez...
Done.
Finish gathering 999 by Selena Gomez
Searching for "Anxiety" by Selena Gomez...
Done.
Finish gathering Anxiety by Selena Gomez
Searching for "Back to You" by Selena Gomez...
Done.
Finish gathering Back to You by Selena Gomez
Searching for "Bad Liar" by Selena Gomez...
Done.
Finish gathering Bad Liar by Selena Gomez
Searching for "Baila Conmigo" by Selena Gomez...
Done.
Finish gathering Baila Conmigo by Selena Gomez
Searching for "Bidi Bidi Bom Bom" by Selena Gomez...
Done.
Finish gathering Bidi Bidi Bom Bom by Selena Gomez
Searching for "Birthday" by Selena Gomez...
Done.
Finish gathering Birthday by Selena Gomez
Searching for "Boyfriend" by Selena Gomez...
Done.
Finish gathering Boyfriend by Selena Gomez
Searching for "Calm Down" by Selena Gomez...
Done.
Finish gathering Calm Down by Selena Gomez
Searching for "Come & Get It" by Selena Gomez...
Done.
Finish gathering Come & Get It by Selena Gomez
Searching

## Quick Wash

In [4]:
input_file=open("lyrics.txt","r")
output_file=open("clean_lyrics.txt","w")

song=[]
begin=False
for line in input_file:
    if line.startswith(">>>"):
        title=line
        print(title)
        continue
    if line.startswith("<sos>"):
        begin=True
        continue
    if line.startswith("<eos>"):
        begin=False
        normal=True
        hyphen_count=0
        for lyric in song:
            if "•" in lyric:
                normal=False
                break
            if "-" in lyric:
                hyphen_count+=1
        if hyphen_count>10:
            normal=False
        if normal:
            prev=None
            output_file.write(title)
            output_file.write("<sos>\n")
            for lyric in song:
                if lyric.startswith("[") or lyric.endswith("]\n"):
                    continue
                if lyric.startswith("(") and lyric.endswith(")\n"):
                    continue
                if lyric=="\n":
                    continue
                if len(lyric.split(" "))<4:
                    continue
                if lyric==prev:
                    continue
                if lyric.endswith("Embed\n"):
                    digit=None
                    for char in lyric:
                        if char.isdigit():
                            digit=char
                            break
                    if digit:
                        index=lyric.index(digit)
                    else:
                        index=lyric.index("E")
                    output_file.write(lyric[:index])
                    output_file.write("\n<eos>\n")
                    
                    song.clear()
                    
                    break
                    
                output_file.write(lyric)
                prev=lyric
        else:
            song.clear()
    if begin:
        song.append(line)

input_file.close()
output_file.close()

>>>The 1 Taylor Swift

>>>22 Taylor Swift

>>>The Alcott Taylor Swift

>>>All of the Girls You Loved Before Taylor Swift

>>>All Too Well Taylor Swift

>>>Anti-Hero Taylor Swift

>>>The Archer Taylor Swift

>>>August Taylor Swift

>>>Babe Taylor Swift

>>>Back to December Taylor Swift

>>>Bad Blood Taylor Swift

>>>Beautiful Ghosts Taylor Swift

>>>Begin Again Taylor Swift

>>>Bejeweled Taylor Swift

>>>The Best Day Taylor Swift

>>>Better Man Taylor Swift

>>>Better than Revenge Taylor Swift

>>>Betty Taylor Swift

>>>Bigger Than the Whole Sky Taylor Swift

>>>Blank Space Taylor Swift

>>>Both of Us Taylor Swift

>>>Breathe Taylor Swift

>>>Call It What You Want Taylor Swift

>>>Cardigan Taylor Swift

>>>Carolina Taylor Swift

>>>Castles Crumbling Taylor Swift

>>>Champagne Problems Taylor Swift

>>>Change Taylor Swift

>>>Christmas Tree Farm Taylor Swift

>>>Clean Taylor Swift

>>>Coney Island Taylor Swift

>>>Cornelia Street Taylor Swift

>>>Crazier Taylor Swift

>>>Cruel Summer Tay


>>>Don't Stop the Music Rihanna

>>>Fading Rihanna

>>>Famous Rihanna

>>>Fly Rihanna

>>>FourFiveSeconds Rihanna

>>>Hard Rihanna

>>>Hate That I Love You Rihanna

>>>Higher Rihanna

>>>If I Never See Your Face Again Rihanna

>>>If It's Lovin' that You Want Rihanna

>>>Jump Rihanna

>>>Just Stand Up! Rihanna

>>>Kiss It Better Rihanna

>>>Lemon Rihanna

>>>Lift Me Up Rihanna

>>>Live Your Life Rihanna

>>>Love on the Brain Rihanna

>>>Love the Way You Lie Rihanna

>>>Love the Way You Lie Rihanna

>>>Loveeeeeee Song Rihanna

>>>Loyalty Rihanna

>>>Man Down Rihanna

>>>The Monster Rihanna

>>>Needed Me Rihanna

>>>Nobody's Business Rihanna

>>>Nothing Is Promised Rihanna

>>>Only Girl Rihanna

>>>Phresh Out the Runway Rihanna

>>>Pon de Replay Rihanna

>>>Pose Rihanna

>>>Pour It Up Rihanna

>>>Princess of China Rihanna

>>>Raining Men Rihanna

>>>Rehab Rihanna

>>>Right Now Rihanna

>>>Rockstar 101 Rihanna

>>>Roll It Gal Rihanna

>>>Rude Boy Rihanna

>>>Run This Town Rihanna

>>>Russ

## remove text within parentheses

In [1]:
import re

In [6]:
input_file=open("clean_lyrics.txt","r")
output_file=open("clean_lyrics_without_parenthese.txt","w")

for line in input_file:
    new_line=re.sub("[\(\[].*?[\)\]]", "", line)
    if new_line !="\n" and new_line:
        output_file.write(new_line)
    
input_file.close()
output_file.close()